In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt

from matplotlib import animation
from matplotlib import rcParams
rcParams['figure.dpi'] = 120
from IPython.display import HTML
from IPython.display import YouTubeVideo
from functools import partial
YouTubeVideo_formato = partial(YouTubeVideo, modestbranding=1, disablekb=0,
                               width=640, height=360, autoplay=0, rel=0, showinfo=0)

# Estadística inferencial: Ajuste de Modelos

La inferencia busca

> Extraer **conclusiones** a partir de **hechos u observaciones** a través de un **método o premisa**

En el caso particular de la **inferencia estadística** podemos realizar las siguientes asociaciones

- Hechos: Datos
- Premisa: Modelo probabilístico
- Conclusión: Una cantidad no observada que es interesante

Y lo que buscamos es

> Cuantificar la incerteza de la conclusión dado los datos y el modelo 

La inferencia estadística puede dividirse en los siguientes tres niveles

1. Ajustar un modelo a nuestros datos
1. Verificar que el modelo sea confiable
1. Responder una pregunta usando el modelo

En las lecciones que siguen estudiaremos las herramientas más utilizadas asociadas a cada uno de estos niveles

1. **Estimador de máxima verosimilitud**
1. **Bondad de ajuste** e **Intervalos de confianza**
1. **Test de hipótesis**

## Ajuste de modelos: Estimación de máxima verosimilitud

En este nivel de inferencia se busca **ajustar** un modelo teórico sobre nuestros datos. En esta lección nos enfocaremos en **modelos de tipo parámetrico**. Un modelo parámetrico es aquel donde **se explicita una distribución de probabilidad**.  

Recordemos que una distribución tiene **parámetros**. Por ejemplo la distribución Gaussiana (univariada) se describe por su media $\mu$ y su varianza $\sigma^2$. Luego ajustar una distribución Gaussiana corresponde a encontrar el valor de $\mu$ y $\sigma$ que hace que el modelo se parezca lo más posible a la distribución empírica de los datos.

A continuación veremos los pasos necesarios para ajustar una distribución a nuestros datos

### ¿Qué distribución ajustar?

Antes de ajustar debemos realizar un supuesto sobre la distribución para nuestro modelo. En general podemos ajustar cualquier distribución pero un mal supuesto podría invalidar nuestra inferencia

Podemos usar las herramientas de **estadística descriptiva** para estudiar nuestros datos y tomar esta decisión de manera informada

En el siguiente ejemplo, un histograma de los datos revela que un modelo gaussiano no es una buena decisión 

<img src="../img/stats6.png">

¿Por qué? La distribución empírica es claramente asimétrica, su cola derecha es más pesada que su cola izquierda. La distribución Gaussiana es simétrica por lo tanto no es apropiada en este caso ¿Qué distribución podría ser más apropiada?



### ¿Cómo ajustar mi modelo? Estimación de máxima verosimilitud

A continuación describiremos un procedimiento para ajustar modelos paramétricos llamado *maximum likelihood estimation* (MLE)

Sea un conjunto de datos $\{x_1, x_2, \ldots, x_N\}$

**Supuesto 1** Los datos siguen el modelo $f(x;\theta)$ donde $f(\cdot)$ es una distribución y $\theta$ son sus parámetros

$$
f(x_1, x_2, \ldots, x_N |\theta)
$$

**Supuesto 2** Las observaciones son independientes e idénticamente distribuidas (iid)

- Si dos variables son independientes se cumple que $P(x, y) = P(x)P(y)$
- Si son además idénticamente distribuidas entonces tienen **la misma distribución y parámetros**

Usando esto podemos escribir

$$
\begin{align}
f(x_1, x_2, \ldots, x_N |\theta) &= f(x_1|\theta) f(x_2|\theta) \ldots f(x_N|\theta) \nonumber \\
& = \prod_{i=1}^N f(x_i|\theta) \nonumber \\
& = \mathcal{L}(\theta)
\end{align}
$$

donde $\mathcal{L}(\theta)$ se conoce como la verosimilitud o probabilidad inversa de $\theta$ 

Si consideramos que los datos son fijos podemos buscar el valor de $\theta$ de máxima verosimilitud

$$
\begin{align}
\hat \theta &= \text{arg} \max_\theta \mathcal{L}(\theta) \nonumber \\
&= \text{arg} \max_\theta \log \mathcal{L}(\theta) \nonumber \\
&= \text{arg} \max_\theta \sum_{i=1}^N \log f(x_i|\theta) 
\end{align}
$$

El segundo paso es valido por que el máximo de $g(x)$ y $\log(g(x))$ es el mismo. El logaritmo es monoticamente creciente. Además aplicar el logaritmo es muy conveniente ya que convierte la multiplicatoria en una sumatoria. 

Ahora sólo falta encontrar el máximo. Podemos hacerlo

- Analíticamente, derivando con respecto a $\theta$ e igualando a cero
- Usando técnicas de optimización iterativas como gradiente descedente

**Ejemplo:** La pesa defectuosa

<img src="../img/garfield.png" width="250">

Su profesor quiere medir su peso pero sospecha que su pesa está defectuosa. Para comprobarlo mide su peso $N$ veces obteniendo un conjunto de observaciones $\{x_i\}$. ¿Es posible obtener un estimador del peso real $\hat x$ a partir de estas observaciones?

Modelaremos las observaciones como

$$
x_i = \hat x + \varepsilon_i
$$

donde $\varepsilon_i$ corresponde al ruido o error del instrumento y asumiremos que $\varepsilon_i \sim \mathcal{N}(0, \sigma_\varepsilon^2)$, es decir que el ruido es **independiente** y **Gaussiano** con media cero y **varianza** $\sigma_\varepsilon^2$ **conocida**

Entonces la distribución de $x_i$ es

$$
f(x_i|\hat x) = \mathcal{N}(\hat x, \sigma_\varepsilon^2)
$$

Para encontrar $\hat x$, primero escribimos el logaritmo de la **verosimilitud**

$$
\begin{align}
\log \mathcal{L}(\hat x) &=  \sum_{i=1}^N \log f(x_i|\hat x) \nonumber \\
&= \sum_{i=1}^N  \log \frac{1}{\sqrt{2\pi\sigma_\varepsilon^2}}  \exp \left ( - \frac{1}{2\sigma_\varepsilon^2} (x_i - \hat x)^2 \right)  \nonumber \\
&= -\frac{N}{2}\log(2\pi\sigma_\varepsilon^2)  - \frac{1}{2\sigma_\varepsilon^2}  \sum_{i=1}^N  (x_i - \hat x)^2  \nonumber
\end{align}
$$

Luego debemos resolver

$$
\begin{align}
\hat \theta &= \text{arg} \max_\theta \log \mathcal{L}(\theta) \nonumber \\
&= \text{arg} \max_\theta - \frac{1}{2\sigma_\varepsilon^2}  \sum_{i=1}^N  (x_i - \hat x)^2
\end{align}
$$

donde podemos ignorar el primer término de la verosimilitud ya que no depende de $\theta$. Para encontrar el máximo derivamos la expresión anterior e igualamos a cero 

$$
-\frac{1}{2\sigma_\varepsilon^2} \sum_{i=1}^N 2(x_i - \hat x ) = 0.
$$

Finalmente si despejamos llegamos a que

$$
\hat x = \frac{1}{N} \sum_{i=1}^N x_i,
$$

que se conoce como el estimador de máxima verosimilitud **para la media de una Gaussiana**

Recordemos que podemos comprobar que es un máximo utilizando la segunda derivada

### Estimación MLE con `scipy`

Como vimos en la lección anterior el módulo [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html) provee de un gran número de distribuciones teóricas organizadas como 

- continuas de una variable
- discretas de una variable
- multivariadas

Las distribuciones comparten muchos de sus métodos, a continuación revisaremos los más importantes. A modo de ejemplo consideremos la distribución Gaussiana (Normal)

```python
from scipy.stats import norm
dist = norm() # Esto crea una Gaussiana con media 0 y desviación estándar (std) 1
dist = norm(loc=2, scale=2) # Esto crea una Gaussiana con media 2 y std 2
```

**Crear una muestra aleatoria con `rvs`**

Luego de crear un objeto distribución podemos obtener una muestra aleatoria usando el método el atributo `rvs` 

```python
dist = norm(loc=2, scale=2)
dist.rvs(size=10, # Cantidad de números aleatorios generados
         random_state=None #Semilla aleatoria
        )
```

Esto retorna un arreglo de 10 números generados aleatoriamente a partir de `dist`

**Evaluar la función de densidad de probabilidad** 

La función de densidad de la Gaussiana es

$$
f(x; \mu, \sigma^2) = \frac{1}{\sqrt{2\pi \sigma^2}} \exp \left( -\frac{1}{2\sigma^2} (x-\mu)^2 \right) 
$$

La densidad de un objeto distribución continuo puede obtenerse con el método `pdf` el cual es función de `x`


```python
dist = norm(loc=2, scale=2)
p = dist.pdf(x # Un ndrray que representa x en la ecuación superior
            )
plt.plot(x, p) # Luego podemos graficar la fdp
```

De forma equivalente, si deseamos la función de densidad acumulada usamos el método `cdf`

Para objetos distribución discretos debemos usar el atributo `pmf` 


**Ajustar los parámetros con MLE**

Para hacer el ajuste se usa el método `fit`

```python 
params = norm.fit(data # Un ndarray con los datos
                 ) 
```

En el caso de la Gaussiana el vector `params` tiene dos componentes `loc` y `scale`. La cantidad de parámetros depende de la distribución que estemos ajustando. También es importante notar que para ajustar se usa `norm` (clase abstracta) y no `norm()` (instancia)

Una vez que tenemos los parámetros ajustados podemos usarlos con

```python
dist = norm(loc=params[0], scale=params[1])
```

Para distribuciones que tienen más de dos parámetros podemos usar

```python
dist = norm(*params[:-2], loc=params[-2], scale=params[-1])
```

### Ejercicio

Observe la siguiente distribución y reflexione ¿Qué características resaltan de la misma? ¿Qué distribución sería apropiado ajustar en este caso?

In [ ]:
df = pd.read_csv('../data/cancer.csv', index_col=0)
df = df[["diagnosis", "radius1", "texture1"]]
x = df["radius1"].values
fig, ax = plt.subplots(figsize=(5, 3), tight_layout=True)
ax.hist(x, bins=20, density=True)
ax.set_xlabel('Radio del nucleo');

- Seleccione una distribución de `scipy.stats`  ajustela a los datos
- Grafique la pdf teórica sobre el histograma

## Verificación de modelos: Tests de bondad de ajuste

Una vez que hemos ajustado un modelo es buena práctica verificar que tan confiable es este ajuste. Las herramientas más típicas para medir que tan bien se ajusta nuestra distribución teórica son

- el [test de Akaike](https://en.wikipedia.org/wiki/Akaike_information_criterion)
- los [gráficos cuantil-cuantil](https://es.wikipedia.org/wiki/Gr%C3%A1fico_Q-Q) (QQ plot)
- el test no-paramétrico de Kolmogorov-Smirnov (KS)

A continuación revisaremos el test de KS para bondad de ajuste

**El test de Kolmogorov-Smirnov**

Es un test no-paramétrico que compara una muestra de datos estandarizados (distribución empírica) con una distribución de densidad acumulada (CDF) teórica. Este test busca refutar la siguiente hipótesis

> **Hipótesis nula:** Las distribuciones son idénticas

Para aplicar el test primero debemos **estandarizar** los datos. Estandarizar se refiere a la transformación

$$
z = \frac{x - \mu_x}{\sigma_x}
$$

es decir los datos estándarizados tienen media cero y desviación estándar uno

Esto puede hacerse fácilmente con NumPy usando

```python
z = (x - np.mean(x))/np.std(x)
```

### Test de KS con `scipy`

Podemos realizar el test de KS con la función [`scipy.stats.kstest`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html) donde

```python
scipy.stats.kstest(rvs, # Una muestra de observaciones estandarizadas
                   cdf, # Una distribución acumulada teórica, por ejemplo scipy.stats.norm.cdf
                   ...
                  )
```

Esta función retorna el valor del estadístico de KS y su *p-value* asociado. Mientras más cerca de cero sea el estadístico de KS mejor es el ajuste. 

Más adelante haremos un repaso de tests de hipótesis en detalle. De momento recordemos que si el *p-value* es menor que una confianza $\alpha=0.05$ entonces rechazamos la hipótesis nula con confianza $1-\alpha = 0.95$ o $95\%$


### Ejercicio 

Considere la muestra de datos anterior
- Seleccione un conjunto de distribuciones teóricas 
- Encuentra la que tiene mejor ajuste usando `kstest`